# 音のプログラミング 第2回: エンベロープとADSR

**学習目標:**
- エンベロープ（音量変化）の概念を理解する
- ADSRエンベロープの4つの段階を学ぶ
- 楽器らしい自然な音を作る
- クリック音の問題と解決方法を体験する

**所要時間:** 90分

## 🛠️ 環境設定

In [4]:
# 🛠️ 環境判定とライブラリセットアップ
import sys
import os

# Google Colab環境かどうかを判定
try:
    import google.colab
    IN_COLAB = True
    print("🔧 Google Colab環境で実行中...")
except ImportError:
    IN_COLAB = False
    print("🏠 ローカル環境で実行中")

# ライブラリのセットアップ
if IN_COLAB:
    print("🔧 Google Colab環境を設定中...")
    
    # 必要なパッケージをインストール
    !pip install numpy scipy matplotlib ipython japanize-matplotlib
    
    # 既存のクローンがある場合は削除（最新版を取得するため）
    !rm -rf /content/simple-audio-programming
    
    # GitHubからライブラリをクローン
    !git clone https://github.com/ggszk/simple-audio-programming.git
    
    # パスを追加
    sys.path.append('/content/simple-audio-programming')
    
    print("✅ セットアップ完了！")
    print("📝 このノートブックを自分用にコピーするには:")
    print("   ファイル → ドライブにコピーを保存")
    
else:
    # ローカル環境では、パッケージが存在するかチェック
    # 変数を事前に初期化して未定義エラーを防ぐ
    current_dir = os.path.dirname(os.path.abspath('.'))
    parent_dir = os.path.dirname(current_dir)
    
    try:
        import audio_lib
        print("audio_libが既にインストールされています")
    except ImportError:
        # パッケージが見つからない場合、パスを追加
        if os.path.exists(os.path.join(parent_dir, 'audio_lib')):
            sys.path.insert(0, parent_dir)
            print(f"パスを追加しました: {parent_dir}")
        else:
            # Poetry環境の場合はパスを直接追加
            project_root = os.path.dirname(current_dir)
            if os.path.exists(os.path.join(project_root, 'audio_lib')):
                sys.path.insert(0, project_root)
                print(f"Poetryプロジェクトのパスを追加しました: {project_root}")
            else:
                # 直接プロジェクトルートを指定（最終フォールバック）
                direct_project_root = '/Users/gsuzuki/projects/teaching/simple-audio-programming'
                if os.path.exists(os.path.join(direct_project_root, 'audio_lib')):
                    sys.path.insert(0, direct_project_root)
                    print(f"プロジェクトルートのパスを追加しました: {direct_project_root}")
                else:
                    print("audio_libディレクトリが見つかりません")

# インポート文（メインモジュールから直接インポート）
try:
    from audio_lib import (
        AudioConfig, SineWave, ADSREnvelope, LinearEnvelope, 
        save_audio, note_to_frequency, frequency_to_note, note_name_to_number
    )
    print("✅ すべての関数を正常にインポートしました")
except ImportError as e:
    print(f"⚠️ インポートエラー: {e}")
    print("🔄 個別インポートを試行中...")
    
    # 基本的なインポート
    from audio_lib import (
        AudioConfig, SineWave, ADSREnvelope, LinearEnvelope, 
        save_audio, note_to_frequency, frequency_to_note
    )
    
    # note_name_to_number を個別インポート
    from audio_lib.synthesis.note_utils import note_name_to_number
    print("✅ 個別インポートで解決しました")

from audio_lib.synthesis.envelopes import apply_envelope
from IPython.display import Audio, display
import numpy as np
import matplotlib.pyplot as plt

# 日本語フォント設定（Colab用）
if IN_COLAB:
    import japanize_matplotlib
    print("✅ 日本語フォントを設定しました")

import warnings
warnings.filterwarnings('ignore')

config = AudioConfig()
sine_osc = SineWave(config)

print("\n🎵 エンベロープとADSRの学習を始めましょう！")

🏠 ローカル環境で実行中
audio_libが既にインストールされています
✅ すべての関数を正常にインポートしました

🎵 エンベロープとADSRの学習を始めましょう！


In [ ]:
# 音声再生用ヘルパー関数
def play_sound(signal, sample_rate=44100, title="Audio"):
    """音声を再生するヘルパー関数"""
    print(f"🔊 {title} (サンプルレート: {sample_rate} Hz)")
    return Audio(signal, rate=sample_rate)

def plot_waveform(signal, sample_rate=44100, duration=0.01, title="波形"):
    """波形を可視化するヘルパー関数"""
    time_samples = int(duration * sample_rate)
    time_samples = min(time_samples, len(signal))
    time_array = np.linspace(0, duration, time_samples)
    
    plt.figure(figsize=(12, 6))
    plt.plot(time_array, signal[:time_samples], 'b-', linewidth=2)
    plt.title(title, fontsize=16)
    plt.xlabel('時間 (秒)', fontsize=12)
    plt.ylabel('振幅', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_envelope_comparison(original, with_envelope, sample_rate=44100, title="エンベロープ比較"):
    """エンベロープ適用前後の比較を可視化"""
    duration = 0.1  # 0.1秒分表示
    samples = int(duration * sample_rate)
    samples = min(samples, len(original), len(with_envelope))
    time_array = np.linspace(0, duration, samples)
    
    plt.figure(figsize=(14, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(time_array, original[:samples], 'r-', linewidth=2, label='エンベロープなし')
    plt.title(f'{title} - エンベロープなし', fontsize=14)
    plt.xlabel('時間 (秒)')
    plt.ylabel('振幅')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(2, 1, 2)
    plt.plot(time_array, with_envelope[:samples], 'b-', linewidth=2, label='エンベロープあり')
    plt.title(f'{title} - エンベロープあり', fontsize=14)
    plt.xlabel('時間 (秒)')
    plt.ylabel('振幅')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.show()

print("🛠️ エンベロープ用ヘルパー関数を読み込みました")

## 🚨 問題: クリック音を体験しよう

前回作ったサイン波には実は問題があります。聞いてみましょう！

In [ ]:
# 前回と同じようにサイン波を作る
frequency = 440
duration = 1.0

raw_signal = sine_osc.generate(frequency, duration)

print("🚨 重要な音響学習ポイント:")
print("📱 Colab/JupyterのAudioは自動正規化するため、音量差やクリック音が軽減されます")
print("🎧 クリック音の違いを明確に聞くにはWAVファイル保存→ダウンロード→再生")
print("=" * 70)

print("🔊 エンベロープなしの音（クリック音に注意！）")
print(f"📊 信号の急激な変化: 開始値={raw_signal[0]:.3f}, 終了値={raw_signal[-1]:.3f}")

# ファイル保存でクリック音を保持
save_audio("signal_without_envelope.wav", config.sample_rate, raw_signal)
print("📁 クリック音確認用: signal_without_envelope.wav を保存しました")

audio_player = play_sound(raw_signal, config.sample_rate, "エンベロープなし（クリック音あり）")
display(audio_player)

## 🎵 エンベロープとは？

### 現実の楽器の特徴
- **ピアノ**: 鍵盤を押した瞬間から音が鳴り、徐々に小さくなる
- **バイオリン**: 弓を当てて徐々に音が大きくなり、弓を離すと消える
- **フルート**: 息を吹き始めてから安定し、息を止めると消える

→ **音量が時間とともに変化する！**

### エンベロープ（包絡線）
音量の時間的変化を制御する仕組み

## 🎯 実習1: シンプルなフェードイン・フェードアウト

In [ ]:
# シンプルなフェードイン・フェードアウトエンベロープ
fade_envelope = LinearEnvelope(
    fade_in=0.1,   # 0.1秒かけてフェードイン
    fade_out=0.1,  # 0.1秒かけてフェードアウト
    config=config
)

# エンベロープデータを生成
envelope_data = fade_envelope.generate(duration)

# エンベロープの形を見てみよう
time_array = np.linspace(0, duration, len(envelope_data))

plt.figure(figsize=(12, 6))
plt.plot(time_array, envelope_data, 'r-', linewidth=3, label='エンベロープ')
plt.title('リニアエンベロープ（フェードイン・フェードアウト）', fontsize=16)
plt.xlabel('時間 (秒)', fontsize=12)
plt.ylabel('音量', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("📊 これがエンベロープの形です！")
print("音の始まりと終わりがなめらかになっています。")

In [ ]:
# エンベロープを音に適用
signal_with_envelope = apply_envelope(raw_signal, envelope_data)

print("🔊 エンベロープ適用後の音")
smooth_audio = Audio(signal_with_envelope, rate=config.sample_rate)
display(smooth_audio)

print("\n✨ クリック音が消えて、自然な音になりました！")

## 🎹 ADSR エンベロープ

楽器の音をもっとリアルに表現するための4段階エンベロープ

### 4つの段階
1. **Attack (アタック)**: 音の立ち上がり
2. **Decay (ディケイ)**: 最大音量から減衰
3. **Sustain (サステイン)**: 一定音量の維持
4. **Release (リリース)**: 音の消失

```
音量
 ↑
 |     /\     
 |    /  \    
 |   /    \___________
 |  /                 \
 | /                   \
 |/                     \
 +--A--D----S------R----→ 時間
```

## 🎯 実習2: ADSRエンベロープを体験しよう

In [ ]:
# 基本的なADSRエンベロープ
adsr = ADSREnvelope(
    attack=0.1,    # 0.1秒でアタック
    decay=0.2,     # 0.2秒でディケイ
    sustain=0.7,   # 70%のレベルでサステイン
    release=0.5,   # 0.5秒でリリース
    config=config
)

duration = 2.0  # 2秒の音
adsr_data = adsr.generate(duration)

# ADSRエンベロープの形を可視化
time_array = np.linspace(0, duration, len(adsr_data))

plt.figure(figsize=(14, 8))
plt.plot(time_array, adsr_data, 'g-', linewidth=3, label='ADSR エンベロープ')

# 各段階に注釈を追加
plt.axvline(x=0.1, color='r', linestyle='--', alpha=0.7, label='Attack終了')
plt.axvline(x=0.3, color='orange', linestyle='--', alpha=0.7, label='Decay終了')
plt.axvline(x=1.5, color='purple', linestyle='--', alpha=0.7, label='Release開始')

plt.title('ADSR エンベロープ', fontsize=16)
plt.xlabel('時間 (秒)', fontsize=12)
plt.ylabel('音量', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print("📊 ADSR の4つの段階が見えますね！")

In [ ]:
# ADSRエンベロープを音に適用
frequency = 440
signal = sine_osc.generate(frequency, duration)
adsr_signal = apply_envelope(signal, adsr_data)

print("🔊 ADSR エンベロープ付きの音")
adsr_audio = Audio(adsr_signal, rate=config.sample_rate)
display(adsr_audio)

print("\n🎹 まるでピアノやシンセサイザーのような音になりました！")

## 🎯 実習3: ADSRパラメータを変えて楽器を模倣しよう

In [ ]:
# ピアノらしい音（速いアタック、長いリリース）
piano_adsr = ADSREnvelope(
    attack=0.01,   # 瞬間的なアタック
    decay=0.3,     # やや長いディケイ
    sustain=0.3,   # 低めのサステイン
    release=1.5,   # 長いリリース
    config=config
)

duration = 3.0
piano_envelope = piano_adsr.generate(duration)
signal = sine_osc.generate(440, duration)
piano_sound = apply_envelope(signal, piano_envelope)

print("🎹 ピアノらしい音")
piano_audio = Audio(piano_sound, rate=config.sample_rate)
display(piano_audio)

In [ ]:
# オルガンらしい音（速いアタック、サステイン重視）
organ_adsr = ADSREnvelope(
    attack=0.05,   # 短いアタック
    decay=0.0,     # ディケイなし
    sustain=1.0,   # 最大レベルでサステイン
    release=0.1,   # 短いリリース
    config=config
)

organ_envelope = organ_adsr.generate(duration)
organ_sound = apply_envelope(signal, organ_envelope)

print("🎺 オルガンらしい音")
organ_audio = Audio(organ_sound, rate=config.sample_rate)
display(organ_audio)

In [ ]:
# 弦楽器らしい音（ゆっくりなアタック）
string_adsr = ADSREnvelope(
    attack=0.3,    # ゆっくりなアタック
    decay=0.1,     # 短いディケイ
    sustain=0.8,   # 高めのサステイン
    release=0.4,   # 中程度のリリース
    config=config
)

string_envelope = string_adsr.generate(duration)
string_sound = apply_envelope(signal, string_envelope)

print("🎻 弦楽器らしい音")
string_audio = Audio(string_sound, rate=config.sample_rate)
display(string_audio)

print("\n💡 同じサイン波でも、エンベロープで全く違う楽器に聞こえます！")

## 🎯 実習4: エンベロープ付きメロディーを作ろう

In [ ]:
# きらきら星のメロディー（最初の部分）
melody_notes = [
    ('C4', 0.5), ('C4', 0.5), ('G4', 0.5), ('G4', 0.5),
    ('A4', 0.5), ('A4', 0.5), ('G4', 1.0),
    ('F4', 0.5), ('F4', 0.5), ('E4', 0.5), ('E4', 0.5),
    ('D4', 0.5), ('D4', 0.5), ('C4', 1.0)
]

# ピアノらしいエンベロープを使用
melody_adsr = ADSREnvelope(
    attack=0.01,
    decay=0.2,
    sustain=0.4,
    release=0.3,
    config=config
)

# メロディーを生成
melody_audio_data = []

for note_name, note_duration in melody_notes:
    # 音名をMIDI番号に変換
    midi_number = note_name_to_number(note_name)
    # MIDI番号を周波数に変換
    frequency = note_to_frequency(midi_number)
    
    # サイン波を生成
    note_signal = sine_osc.generate(frequency, note_duration)
    
    # エンベロープを適用
    envelope_data = melody_adsr.generate(note_duration)
    note_with_envelope = apply_envelope(note_signal, envelope_data)
    
    melody_audio_data.append(note_with_envelope)

# 全ての音をつなげる
full_melody = np.concatenate(melody_audio_data)

print("🌟 きらきら星（エンベロープ付き）")
melody_audio = Audio(full_melody, rate=config.sample_rate)
display(melody_audio)

print("\n🎉 美しいメロディーができました！")
print("エンベロープのおかげで、各音符が自然につながっています。")

## 🎯 実習5: エンベロープありなしの比較

In [ ]:
# 同じメロディーをエンベロープなしで作る
melody_without_envelope = []

for note_name, note_duration in melody_notes:
    midi_number = note_name_to_number(note_name)
    frequency = note_to_frequency(midi_number)
    note_signal = sine_osc.generate(frequency, note_duration)
    melody_without_envelope.append(note_signal)

melody_raw = np.concatenate(melody_without_envelope)

print("🚨 エンベロープなしのメロディー（クリック音注意）")
raw_melody_audio = Audio(melody_raw, rate=config.sample_rate)
display(raw_melody_audio)

print("\n✨ エンベロープありのメロディー")
display(melody_audio)

print("\n💡 比較してみてください：")
print("- エンベロープなし：各音符の境界でクリック音")
print("- エンベロープあり：なめらかで自然な音楽")

## 🏆 チャレンジ課題

In [ ]:
# チャレンジ1: あなただけのエンベロープを作ろう
# パラメータを変更して、理想の音を見つけてください

my_adsr = ADSREnvelope(
    attack=0.1,     # ここを変更してください (0.01～1.0)
    decay=0.2,      # ここを変更してください (0.0～1.0)
    sustain=0.5,    # ここを変更してください (0.0～1.0)
    release=0.3,    # ここを変更してください (0.1～2.0)
    config=config
)

# テスト音を作成
test_duration = 2.0
test_signal = sine_osc.generate(440, test_duration)
my_envelope = my_adsr.generate(test_duration)
my_sound = apply_envelope(test_signal, my_envelope)

# エンベロープの形を表示
time_array = np.linspace(0, test_duration, len(my_envelope))
plt.figure(figsize=(12, 6))
plt.plot(time_array, my_envelope, 'b-', linewidth=3)
plt.title('あなたのオリジナルエンベロープ', fontsize=16)
plt.xlabel('時間 (秒)')
plt.ylabel('音量')
plt.grid(True, alpha=0.3)
plt.show()

print("🎵 あなたのオリジナル音色")
my_audio = Audio(my_sound, rate=config.sample_rate)
display(my_audio)

In [ ]:
# チャレンジ2: 短いメロディーを作ろう
# 好きな音符を並べてください

# 使える音名: C4, D4, E4, F4, G4, A4, B4, C5
my_melody = [
    ('C4', 0.5),   # ド
    ('E4', 0.5),   # ミ
    ('G4', 0.5),   # ソ
    ('C5', 1.0),   # 高いド
    # ここに音符を追加してください
]

# あなたのエンベロープを使用
my_melody_audio = []

for note_name, note_duration in my_melody:
    midi_number = note_name_to_number(note_name)
    frequency = note_to_frequency(midi_number)
    note_signal = sine_osc.generate(frequency, note_duration)
    envelope_data = my_adsr.generate(note_duration)
    note_with_envelope = apply_envelope(note_signal, envelope_data)
    my_melody_audio.append(note_with_envelope)

my_full_melody = np.concatenate(my_melody_audio)

print("🎵 あなたのオリジナルメロディー")
my_melody_final = Audio(my_full_melody, rate=config.sample_rate)
display(my_melody_final)

print("\n🎉 素晴らしい！あなただけの音楽です！")

## 📚 今日のまとめ

### 学んだこと
1. **クリック音の問題**: 音の急な開始・終了で発生
2. **エンベロープ**: 音量の時間的変化を制御
3. **ADSR**: Attack, Decay, Sustain, Release の4段階
4. **楽器の特徴**: エンベロープで楽器らしさを表現
5. **音楽的表現**: メロディーに自然さを与える

### 使ったライブラリ
- `LinearEnvelope`: シンプルなフェード
- `ADSREnvelope`: 4段階エンベロープ
- `apply_envelope`: エンベロープの適用

### エンベロープの効果
- **技術的**: クリック音の除去
- **音楽的**: 楽器らしい表現
- **芸術的**: 音の感情表現

### 次回予告
次回は「**オシレーターと音色**」を学びます。
サイン波以外の波形（ノコギリ波、矩形波など）を使って、もっと豊かな音色を作ります！

---
**お疲れさまでした！** 🎉